# 🦺 Intelligent PPE Monitoring System — Colab Test

**Purpose:** Test the full YOLO + SAM hybrid detection pipeline on a free GPU before Azure deployment.

## Pipeline Overview
```
Image → YOLO (1280px) → 5-Path Decision → SAM (ROI crop) → Result
         ~35ms                              ~1500ms async
```

## Before Running
1. **Runtime → Change runtime type → T4 GPU** ✅
2. Upload your models to Google Drive:
   - `MyDrive/ppe_models/best.pt` (your YOLO model)
   - `MyDrive/ppe_models/sam3.pt` (SAM model)
3. Run cells **top to bottom** in order

---

## Cell 1: Verify GPU

In [ ]:
import torch

print('='*50)
print('GPU CHECK')
print('='*50)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'✅ GPU: {gpu_name}')
    print(f'✅ VRAM: {gpu_mem:.1f} GB')
else:
    print('❌ No GPU found!')
    print('   Go to Runtime → Change runtime type → T4 GPU')
    raise RuntimeError('GPU required for SAM. Please enable GPU runtime.')

print(f'✅ PyTorch: {torch.__version__}')
print(f'✅ CUDA: {torch.version.cuda}')

## Cell 2: Install Dependencies

In [ ]:
print('Installing dependencies...')

# Core ML
!pip install -q ultralytics>=8.0.0

# API server + tunnel
!pip install -q fastapi uvicorn[standard] python-multipart pyngrok

# Database + config
!pip install -q sqlalchemy pydantic>=2.0.0 pydantic-settings python-dotenv

# Image processing
!pip install -q opencv-python-headless Pillow

# Reporting
!pip install -q reportlab apscheduler

# Download Sam 3
#!wget --header="Authorization: Bearer YOUR_HF_TOKEN" "https://huggingface.co/facebook/sam3/resolve/main/sam3.pt"

print('\n✅ All dependencies installed!')

## Cell 3: Mount Google Drive & Copy Models

In [ ]:
from google.colab import drive
import os, shutil

drive.mount('/content/drive')

# ── Configure your Drive paths here ──────────────────────────────
DRIVE_YOLO_MODEL = '/content/drive/MyDrive/ppe_models/best.pt'
DRIVE_SAM_MODEL  = '/content/drive/MyDrive/ppe_models/sam3.pt'
# ─────────────────────────────────────────────────────────────────

# Create models directory
os.makedirs('/content/backend/models', exist_ok=True)

# Copy models (faster than loading from Drive every time)
print('Copying models from Drive...')

if os.path.exists(DRIVE_YOLO_MODEL):
    shutil.copy(DRIVE_YOLO_MODEL, '/content/backend/models/best.pt')
    size_mb = os.path.getsize('/content/backend/models/best.pt') / 1e6
    print(f'✅ YOLO model copied ({size_mb:.1f} MB)')
else:
    print(f'❌ YOLO model not found at: {DRIVE_YOLO_MODEL}')
    print('   Please upload best.pt to your Google Drive')

if os.path.exists(DRIVE_SAM_MODEL):
    shutil.copy(DRIVE_SAM_MODEL, '/content/backend/models/sam3.pt')
    size_mb = os.path.getsize('/content/backend/models/sam3.pt') / 1e6
    print(f'✅ SAM model copied ({size_mb:.1f} MB)')
else:
    print(f'⚠️  SAM model not found at: {DRIVE_SAM_MODEL}')
    print('   System will run in YOLO-only mode (SAM disabled)')

## Cell 4: Clone / Upload Backend Code

**Option A:** Clone from GitHub (if you pushed your code)

**Option B:** Upload the backend folder as a zip (manual)

Run whichever applies to you:

In [ ]:
import os

# ── OPTION A: Clone from GitHub ───────────────────────────────────
# Uncomment and fill in your repo URL:
# !git clone https://github.com/YOUR_USERNAME/intelligent-ppe-monitoring.git /content/repo
# !cp -r /content/repo/backend/* /content/backend/

# ── OPTION B: Copy from Google Drive ─────────────────────────────
# If you zipped your backend folder and uploaded to Drive:
# DRIVE_BACKEND_ZIP = '/content/drive/MyDrive/ppe_models/backend.zip'
# !unzip -q {DRIVE_BACKEND_ZIP} -d /content/backend_extracted
# !cp -r /content/backend_extracted/backend/* /content/backend/

# ── OPTION C: Write backend inline (auto-setup) ───────────────────
# This creates the minimum files needed to run detection
# Use this if you don't have GitHub or Drive zip

print('Choose your option above and uncomment the relevant lines.')
print('Then re-run this cell.')
print()
print('If using Option C (inline), run the next cell instead.')

## Cell 4b: Auto-Setup Backend (if no GitHub/Drive zip)

In [ ]:
import os

# Create full directory structure
dirs = [
    '/content/backend',
    '/content/backend/config',
    '/content/backend/database',
    '/content/backend/services',
    '/content/backend/agents',
    '/content/backend/api',
    '/content/backend/api/routes',
    '/content/backend/api/models',
    '/content/backend/utils',
    '/content/backend/models',
    '/content/backend/uploads',
    '/content/backend/reports',
]
for d in dirs:
    os.makedirs(d, exist_ok=True)

# Create __init__.py files
init_dirs = ['config','database','services','agents','api','api/routes','api/models','utils']
for d in init_dirs:
    open(f'/content/backend/{d}/__init__.py', 'w').close()

print('✅ Directory structure created')
print()
print('⚠️  You still need to copy your Python source files.')
print('   Best approach: zip your backend folder, upload to Drive,')
print('   then use Option B in Cell 4.')

## Cell 5: Configure Environment

In [ ]:
import os

# Write .env file for the backend
env_content = """
# Database
DATABASE_URL=sqlite:////content/backend/ppe_detection.db

# YOLO Model
YOLO_MODEL_PATH=/content/backend/models/best.pt
YOLO_CONFIDENCE_THRESHOLD=0.25
YOLO_IMGSZ=1280

# SAM Model
SAM_ENABLED=true
SAM_MODEL_PATH=/content/backend/models/sam3.pt
SAM_DEVICE=cuda
SAM_MASK_THRESHOLD=0.05

# Violation Tracking
VIOLATION_COOLDOWN_SECONDS=300
VIOLATION_IOU_THRESHOLD=0.3
VIOLATION_TRACK_TIMEOUT=60

# Site defaults
DEFAULT_SITE_LOCATION=Colab Test Site
DEFAULT_CAMERA_ID=colab_cam_01

# Reporting
REPORT_OUTPUT_DIR=/content/backend/reports
REPORT_TIME=23:59

# Debug
DEBUG=true
""".strip()

with open('/content/backend/.env', 'w') as f:
    f.write(env_content)

print('✅ .env file written')

# Set working directory
os.chdir('/content/backend')
print(f'✅ Working directory: {os.getcwd()}')

## Cell 6: Initialize Database

In [ ]:
import sys
sys.path.insert(0, '/content/backend')
os.chdir('/content/backend')

from database.connection import engine
from database.models import Base

# Create all tables
Base.metadata.create_all(bind=engine)
print('✅ Database tables created')

# List tables
from sqlalchemy import inspect
inspector = inspect(engine)
tables = inspector.get_table_names()
print(f'✅ Tables: {tables}')

## Cell 7: Load Models & Test Detection (No API)

In [ ]:
import sys, os, time
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

sys.path.insert(0, '/content/backend')
os.chdir('/content/backend')

# ── Load YOLO ────────────────────────────────────────────────────
print('Loading YOLO model...')
from services.yolo_detector import get_yolo_detector
yolo = get_yolo_detector()
yolo.load_model()
print('✅ YOLO loaded')

# ── Load SAM ─────────────────────────────────────────────────────
print('Loading SAM model...')
from services.sam_verifier import get_sam_verifier
sam = get_sam_verifier()
sam.load_model()
if sam._use_mock:
    print('⚠️  SAM running in MOCK mode (model not found or load failed)')
else:
    print('✅ SAM loaded (real GPU inference)')

# ── Create test image ────────────────────────────────────────────
# Use a blank image for smoke test (replace with real image below)
test_image = np.zeros((720, 1280, 3), dtype=np.uint8)
test_image[:] = (50, 50, 50)  # Dark gray background

# ── Run YOLO detection ───────────────────────────────────────────
print('\nRunning YOLO detection on test image...')
t0 = time.time()
yolo_result = yolo.detect(test_image)
yolo_ms = (time.time() - t0) * 1000

print(f'✅ YOLO inference: {yolo_ms:.1f}ms')
print(f'   Persons detected: {len(yolo_result["persons"])}')
print(f'   PPE items detected: {len(yolo_result["ppe_items"])}')
print()
print('Smoke test passed! Ready to test with real images.')

## Cell 8: Test with a Real Image

Upload a construction site image and run the full hybrid detection pipeline.

In [ ]:
from google.colab import files
import io

print('Upload a test image (JPG/PNG)...')
uploaded = files.upload()

# Load the uploaded image
filename = list(uploaded.keys())[0]
image_data = uploaded[filename]
nparr = np.frombuffer(image_data, np.uint8)
test_image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

print(f'✅ Image loaded: {filename} ({test_image.shape[1]}×{test_image.shape[0]}px)')

In [ ]:
import sys, os
sys.path.insert(0, '/content/backend')
os.chdir('/content/backend')

import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# ── Run full hybrid detection ─────────────────────────────────────
print('Running full hybrid detection pipeline...')
print('='*50)

from services.hybrid_detector import get_hybrid_detector
detector = get_hybrid_detector()

t0 = time.time()
result = detector.detect_async(
    test_image,
    save_annotated=True,
    output_path='/content/backend/uploads/test_annotated.jpg'
)
total_ms = (time.time() - t0) * 1000

# ── Print results ─────────────────────────────────────────────────
print(f'\n⏱️  Total time: {total_ms:.1f}ms')
print(f'   YOLO time:  {result["timing"]["yolo_ms"]:.1f}ms')
print(f'   SAM mode:   {result.get("sam_mode", "sync")}')
print(f'   SAM jobs:   {result.get("sam_jobs_pending", 0)} pending in background')
print()
print(f'👥 Persons detected: {result["stats"]["total_persons"]}')
print(f'🚨 Violations:       {result["stats"]["total_violations"]}')
print(f'✅ Compliance rate:  {result["stats"]["compliance_rate"]:.1f}%')
print(f'🔬 SAM activations:  {result["stats"]["sam_activations"]}')
print()

for i, person in enumerate(result['persons']):
    status = '🚨 VIOLATION' if person['is_violation'] else '✅ SAFE'
    print(f'  Person {i+1}: {status}')
    print(f'    Helmet: {"✅" if person["has_helmet"] else "❌"}')
    print(f'    Vest:   {"✅" if person["has_vest"] else "❌"}')
    print(f'    Path:   {person["decision_path"]}')
    print(f'    SAM:    {"Yes" if person["sam_activated"] else "No"}')
    if person['violation_type']:
        print(f'    Type:   {person["violation_type"]}')
    print()

# ── Visualize ─────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Original image with bboxes
img_rgb = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
axes[0].imshow(img_rgb)
axes[0].set_title('Original + Detections', fontsize=14)

for person in result['persons']:
    x1, y1, x2, y2 = person['bbox']
    color = 'red' if person['is_violation'] else 'green'
    rect = patches.Rectangle(
        (x1, y1), x2-x1, y2-y1,
        linewidth=2, edgecolor=color, facecolor='none'
    )
    axes[0].add_patch(rect)
    label = f'{person["decision_path"]}'
    axes[0].text(x1, y1-5, label, color=color, fontsize=9, fontweight='bold')

axes[0].axis('off')

# Annotated image (if saved)
annotated_path = '/content/backend/uploads/test_annotated.jpg'
if os.path.exists(annotated_path):
    ann_img = cv2.imread(annotated_path)
    ann_rgb = cv2.cvtColor(ann_img, cv2.COLOR_BGR2RGB)
    axes[1].imshow(ann_rgb)
    axes[1].set_title('Annotated Output', fontsize=14)
else:
    axes[1].text(0.5, 0.5, 'No annotated image', ha='center', va='center')
axes[1].axis('off')

plt.tight_layout()
plt.savefig('/content/detection_result.png', dpi=150, bbox_inches='tight')
plt.show()
print('\n✅ Result saved to /content/detection_result.png')

## Cell 9: Wait for SAM Results & Check Accuracy

In [ ]:
import time
from services.async_sam_verifier import get_async_sam_verifier

async_sam = get_async_sam_verifier()
sam_jobs = result.get('sam_jobs', [])

if not sam_jobs:
    print('ℹ️  No SAM jobs were submitted (all persons took fast paths)')
else:
    print(f'⏳ Waiting for {len(sam_jobs)} SAM job(s) to complete...')
    
    for job_id in sam_jobs:
        sam_result = async_sam.wait_for(job_id, timeout=60.0)
        if sam_result:
            print(f'\n✅ SAM Job {job_id[:8]}...')
            print(f'   Helmet: {"✅" if sam_result.has_helmet else "❌"}')
            print(f'   Vest:   {"✅" if sam_result.has_vest else "❌"}')
            print(f'   Is violation: {sam_result.is_violation}')
            print(f'   YOLO was correct: {sam_result.yolo_was_correct}')
            print(f'   SAM latency: {sam_result.sam_latency_ms:.1f}ms')
            if not sam_result.yolo_was_correct:
                print(f'   🔄 SAM CORRECTED YOLO!')
        else:
            print(f'⚠️  Job {job_id[:8]}... timed out')

# Print SAM statistics
print('\n' + '='*50)
print('SAM STATISTICS (Thesis Metrics)')
print('='*50)
stats = async_sam.get_stats()
for key, val in stats.items():
    print(f'  {key}: {val}')

## Cell 10: Start FastAPI Server with ngrok

This exposes your backend as a public URL so your frontend can connect to it.

**Get a free ngrok token at:** https://dashboard.ngrok.com/signup

In [ ]:
# ── Set your ngrok token ──────────────────────────────────────────
NGROK_TOKEN = 'YOUR_NGROK_TOKEN_HERE'  # ← paste your token here
# ─────────────────────────────────────────────────────────────────

import subprocess, threading, time
from pyngrok import ngrok, conf

# Configure ngrok
conf.get_default().auth_token = NGROK_TOKEN

# Kill any existing tunnels
ngrok.kill()

# Start FastAPI in background thread
def run_server():
    import subprocess
    subprocess.run(
        ['python', '-m', 'uvicorn', 'main:app', '--host', '0.0.0.0', '--port', '8000'],
        cwd='/content/backend'
    )

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start
print('Starting FastAPI server...')
time.sleep(8)

# Create ngrok tunnel
tunnel = ngrok.connect(8000)
public_url = tunnel.public_url

print('='*60)
print('🌐 PUBLIC API URL:')
print(f'   {public_url}')
print()
print('📋 API Endpoints:')
print(f'   Health:    {public_url}/api/health')
print(f'   Detect:    {public_url}/api/detect  (POST)')
print(f'   History:   {public_url}/api/history')
print(f'   SAM Stats: {public_url}/api/sam/stats')
print(f'   API Docs:  {public_url}/docs')
print('='*60)
print()
print('👉 Copy this URL into your frontend .env as VITE_API_URL')
print(f'   VITE_API_URL={public_url}')

## Cell 11: Test API via HTTP (while server is running)

In [ ]:
import requests, json

# Test health endpoint
print('Testing /api/health...')
r = requests.get(f'{public_url}/api/health')
print(f'Status: {r.status_code}')
print(json.dumps(r.json(), indent=2))

In [ ]:
# Test detection endpoint with your uploaded image
print('Testing /api/detect...')

with open('/content/backend/uploads/test_annotated.jpg', 'rb') as f:
    # Use original image if annotated doesn't exist
    pass

# Re-encode test image
import cv2
_, img_encoded = cv2.imencode('.jpg', test_image)

r = requests.post(
    f'{public_url}/api/detect',
    files={'file': ('test.jpg', img_encoded.tobytes(), 'image/jpeg')},
    data={'save_annotated': 'true'}
)

print(f'Status: {r.status_code}')
result_api = r.json()
print(f'Message: {result_api.get("message")}')
print(f'Persons: {result_api["stats"]["total_persons"]}')
print(f'Violations: {result_api["stats"]["total_violations"]}')
print(f'Compliance: {result_api["stats"]["compliance_rate"]:.1f}%')

## Cell 12: Benchmark — FPS & Latency Test

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from services.yolo_detector import get_yolo_detector

yolo = get_yolo_detector()
N_RUNS = 20

print(f'Running {N_RUNS} inference passes for benchmarking...')

latencies = []
for i in range(N_RUNS):
    t0 = time.time()
    yolo.detect(test_image)
    ms = (time.time() - t0) * 1000
    latencies.append(ms)
    if (i+1) % 5 == 0:
        print(f'  Run {i+1}/{N_RUNS}: {ms:.1f}ms')

# Skip first 3 warmup runs
latencies_warm = latencies[3:]

avg_ms = np.mean(latencies_warm)
min_ms = np.min(latencies_warm)
max_ms = np.max(latencies_warm)
fps = 1000 / avg_ms

print()
print('='*50)
print('YOLO BENCHMARK RESULTS (Thesis Table)')
print('='*50)
print(f'  Average latency: {avg_ms:.1f}ms')
print(f'  Min latency:     {min_ms:.1f}ms')
print(f'  Max latency:     {max_ms:.1f}ms')
print(f'  FPS:             {fps:.1f}')
print(f'  Image size:      1280×1280')
print(f'  Device:          {"GPU" if __import__("torch").cuda.is_available() else "CPU"}')

# Plot
plt.figure(figsize=(10, 4))
plt.plot(latencies_warm, marker='o', linewidth=2, color='#2196F3')
plt.axhline(avg_ms, color='red', linestyle='--', label=f'Avg: {avg_ms:.1f}ms')
plt.xlabel('Run #')
plt.ylabel('Latency (ms)')
plt.title(f'YOLO Inference Latency — {fps:.1f} FPS')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/content/benchmark_yolo.png', dpi=150)
plt.show()
print('\n✅ Benchmark chart saved to /content/benchmark_yolo.png')

## Cell 13: Download Results

In [ ]:
from google.colab import files
import os

# Download detection result
if os.path.exists('/content/detection_result.png'):
    files.download('/content/detection_result.png')
    print('✅ Downloaded: detection_result.png')

# Download benchmark chart
if os.path.exists('/content/benchmark_yolo.png'):
    files.download('/content/benchmark_yolo.png')
    print('✅ Downloaded: benchmark_yolo.png')

# Download annotated image
ann_path = '/content/backend/uploads/test_annotated.jpg'
if os.path.exists(ann_path):
    files.download(ann_path)
    print('✅ Downloaded: test_annotated.jpg')

print('\n📊 Use these images in your thesis!')

---

## ✅ Checklist

- [ ] GPU enabled (T4)
- [ ] `best.pt` loaded successfully
- [ ] `sam3.pt` loaded successfully (real, not mock)
- [ ] YOLO detection works on test image
- [ ] SAM async jobs complete
- [ ] API server running with ngrok URL
- [ ] Frontend can connect to ngrok URL
- [ ] FPS benchmark recorded for thesis

## 🎓 Thesis Metrics to Record

| Metric | Value |
|--------|-------|
| YOLO inference time (ms) | __ |
| YOLO FPS | __ |
| SAM latency (ms) | __ |
| False positives caught by SAM | __ |
| False negatives caught by SAM | __ |
| YOLO accuracy rate | __ % |
| SAM activation rate | __ % |

Fill these in from Cell 9 (SAM stats) and Cell 12 (benchmark).